# Discount Impact
- We want to briefly analyze the impact of offering a 20% discount to customers who are likely to churn as suggested by the SME division head
- Since PowerCo's pricing structure is not very clear we approximate revenue with forecast_price_energy_off_peak (price) * forecast_cons_12m (amount)
- The pricing structure is probably much more complex and would need to be discussed with PowerCo for a more complete analysis

In [1]:
import pandas as pd

In [2]:
# Load out of sample test data with churn predictions
df = pd.read_csv("test_predictions.csv")

## Revenue with no discount

In [3]:
# Revenue if no churn
df["revenue"] = df["forecast_price_energy_off_peak"] * df["forecast_cons_12m"]
# Churn corresponds to the customer churning in the next 3 months
# We use 01.01.2016 as current date
# The revenue for a churning customer is therefore approximated as revenue over the next 12 months * 3/12
df["revenue_churn"] = df["revenue"] * 3/12
# compute the actual revenue
df.apply(lambda row: row["revenue"] if row["churn"]==0 else row["revenue_churn"], axis=1).sum().round(2)

645211.52

- The approximate revenue for the next your without intervention is 645211

## Revenue with discount
- If the model predicts that a customer is likely to churn, the customer is offered a 20% discount
- *Assumption*: Any customer who is offered a discount will accept the offer and not churn

In [4]:
def get_revenue(row):
    # no churn predcited and client does not churn
    if row["churn"] == 0 and row["churn_prediction"] == 0:
        return row["revenue"]
    # no churn predicted but client churns
    elif row["churn"] == 1 and row["churn_prediction"] == 0:
        return row["revenue_churn"]
    # churn predicted and client is offered discount
    else:
        return row["revenue_discount"]
    
discount = 0.2
df["revenue_discount"] = df["revenue"] * (1 - discount)
df.apply(lambda row: get_revenue(row), axis=1).sum().round(2)

646031.37

- The approximate revenue for the next year offering a 20% discount to customers for whom the models predict churn is 646031
- This is slightly higher than the revenue without the intervention
- However, bote that this is only a very simplified analysis

## Next steps
- Different discount percentages can be compared to maximize expected revenue
- A different cutoff for churn probability can be used, currently a customer is predicted to churn if the churn probability is > 0.5